In [21]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
import pandas as pd
import numpy as np
import time
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import pickle

# filter out warnings
import warnings
warnings.filterwarnings('ignore')

In [28]:
df = pd.read_csv('../data/final/fta_pop_gdp_cleaned_v2.csv')
df_trades=pd.read_csv('../data/final/trades_v3_combined.csv')
df_products=pd.read_csv('../data/final/products.csv')
df['country_code'] = df['country_code'].map(lambda x: x.lower())

In [33]:
df_fta = df[df['has_fta']][['country', 'country_code', 'signed_date', 'date_inforce']]
df_fta['signed_date'] = pd.to_datetime(df_fta['signed_date'])
df_fta['date_inforce'] = pd.to_datetime(df_fta['date_inforce'])
df_fta['year_inforce'] = df_fta['date_inforce'].map(lambda x: x.year)
df_fta['country_code'] = df_fta['country_code'].map(lambda x: x.lower())

In [34]:
# 1 if usa is seller, else 0
df_trades['seller_usa'] = df_trades.apply(lambda x: 1 if x['exports_from']=='usa' else 0, axis=1)
df_trades['trade_country'] = df_trades.apply(lambda x: x['exports_to'] if x['exports_from']=='usa' else x['exports_from'], axis=1)
df_train = df_trades.pivot_table(index = ['trade_country', 'seller_usa', 'exports_year'], values = ['trade_value'], aggfunc='sum').reset_index()
df_train['fta_inforce'] = df_train.merge(df_fta[['country_code', 'year_inforce']], how='left', left_on='trade_country', right_on='country_code')\
.apply(lambda x: 1 if x['year_inforce']==x['year_inforce'] and x['year_inforce'] <= x['exports_year'] else 0, axis=1)
df_train['train'] = True
df_train

,trade_country,seller_usa,exports_year,trade_value,fta_inforce,train
0,abw,0,2000,1.336525e+09,0,True
1,abw,0,2001,9.364850e+08,0,True
2,abw,0,2002,7.312244e+08,0,True
3,abw,0,2003,9.114637e+08,0,True
4,abw,0,2004,1.702326e+09,0,True
...,...,...,...,...,...,...
7733,zwe,1,2015,6.956188e+07,0,True
7734,zwe,1,2016,6.388521e+07,0,True
7735,zwe,1,2017,4.677972e+07,0,True
7736,zwe,1,2018,4.620724e+07,0,True


In [8]:
fta_year = 2010
counter_country = 'deu'
df_test = df_train[df_train['trade_country']==counter_country]
df_test['fta_inforce'] = df_test['exports_year'].map(lambda x: 1 if x >= fta_year else 0)
df_test['train'] = False

In [45]:
df_train['fta_inforce'].value_counts()

1    7738
0    7265
Name: fta_inforce, dtype: int64

In [36]:
df_train = pd.concat([df_train, 
                    resample(df_train[df_train['fta_inforce']==1], replace=True, n_samples=df_train.shape[0]-df_train['fta_inforce'].value_counts()[1])],
                    axis=0)

In [ ]:
df_train['fta_inforce'].value_counts()

1    7738
0    7265
Name: fta_inforce, dtype: int64

In [42]:
pickle.dump(df_train, open('../data/final/traindata.pkl','wb'))

In [37]:
def get_X (train, test):
  df = pd.concat([train, test], axis=0)
  df_all = pd.concat([pd.get_dummies(df[['trade_country']].astype(str), drop_first=True),
               df[['seller_usa', 'fta_inforce', 'exports_year', 'train']]], axis=1)
  return df_all[df_all['train']==True].iloc[:,:-1], df_all[df_all['train']==False].iloc[:,:-1], train['trade_value'], test['trade_value']

In [38]:
X_train, X_test, y_train, y_stale = get_X(df_train, df_test)

In [39]:
X_train.shape, X_test.shape, len(y_train), len(y_stale)

((15003, 220), (42, 220), 15003, 42)

In [40]:
def show_prediction(df, pred, counter_country):
  full = df[['seller_usa', 'exports_year', 'trade_value']].copy()
  full['pred'] = pred
  full['pred'] = full.apply(lambda x: x['pred'] if x['exports_year']>=fta_year else x['trade_value'], axis=1)
  full['status'] = 'Non-FTA'
  result = full[['seller_usa', 'exports_year', 'trade_value', 'status']]
  full['status'] = 'If FTA in Force'
  full['trade_value'] = full['pred'] 
  result = pd.concat([result, full[['seller_usa', 'exports_year', 'trade_value', 'status']]], axis=0)
  
  fig = px.line(result[result['seller_usa']==0], x='exports_year', y='trade_value', color='status', title='USA sells '+counter_country.upper()+' buys')
  fig.show()

  fig = px.line(result[result['seller_usa']==1], x='exports_year', y='trade_value', color='status', title='USA buys '+counter_country.upper()+' sells')
  fig.show()

In [17]:
lr_cv = GridSearchCV(LinearRegression(), {'normalize': [True, False]}, cv=5, verbose=3)
lr_cv.fit(X_train, y_train)
show_prediction(df_test, lr_cv.best_estimator_.predict(X_test), counter_country)
print(lr_cv.best_estimator_)
print(lr_cv.best_params_)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END normalize=True;, score=-38789462228233205847188045824.000 total time=   0.4s
[CV 2/5] END normalize=True;, score=-80480320165798357903144386560.000 total time=   0.5s
[CV 3/5] END ....................normalize=True;, score=0.934 total time=   0.5s
[CV 4/5] END ....................normalize=True;, score=0.942 total time=   0.6s
[CV 5/5] END ....................normalize=True;, score=0.946 total time=   0.6s
[CV 1/5] END normalize=False;, score=-14144436497274778943488.000 total time=   0.4s
[CV 2/5] END ...................normalize=False;, score=0.547 total time=   0.4s
[CV 3/5] END ...................normalize=False;, score=0.934 total time=   0.4s
[CV 4/5] END ...................normalize=False;, score=0.942 total time=   0.3s
[CV 5/5] END ...................normalize=False;, score=0.946 total time=   0.3s


LinearRegression(normalize=False)
{'normalize': False}


In [18]:
%%time
Rdg_cv = GridSearchCV(Ridge(), {'alpha':[1e-3, 1e-2, 1e-1, 1, 10, 100]}, cv=5, verbose=3)
Rdg_cv.fit(X_train, y_train)
show_prediction(df_test, Rdg_cv.best_estimator_.predict(X_test), counter_country)
print(Rdg_cv.best_estimator_)
print(Rdg_cv.best_params_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END .......................alpha=0.001;, score=0.470 total time=   0.4s
[CV 2/5] END .......................alpha=0.001;, score=0.548 total time=   0.4s
[CV 3/5] END .......................alpha=0.001;, score=0.934 total time=   0.6s
[CV 4/5] END .......................alpha=0.001;, score=0.942 total time=   0.6s
[CV 5/5] END .......................alpha=0.001;, score=0.946 total time=   0.5s
[CV 1/5] END ........................alpha=0.01;, score=0.470 total time=   0.6s
[CV 2/5] END ........................alpha=0.01;, score=0.553 total time=   0.5s
[CV 3/5] END ........................alpha=0.01;, score=0.934 total time=   0.6s
[CV 4/5] END ........................alpha=0.01;, score=0.942 total time=   0.3s
[CV 5/5] END ........................alpha=0.01;, score=0.946 total time=   0.6s
[CV 1/5] END .........................alpha=0.1;, score=0.470 total time=   0.6s
[CV 2/5] END .........................alpha=0.1;,

Ridge(alpha=0.1)
{'alpha': 0.1}
CPU times: user 1min 16s, sys: 30.6 s, total: 1min 46s
Wall time: 15.9 s


In [19]:
%%time
lso_cv = GridSearchCV(Lasso(), {'alpha':[1e-3, 1e-2, 1e-1, 1, 10, 100]}, cv=5, verbose=3)
lso_cv.fit(X_train, y_train)
show_prediction(df_test, lso_cv.best_estimator_.predict(X_test), counter_country)
print(lso_cv.best_estimator_)
print(lso_cv.best_score_)
print(lso_cv.best_params_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END .......................alpha=0.001;, score=0.443 total time=   8.3s
[CV 2/5] END .......................alpha=0.001;, score=0.551 total time=   7.9s
[CV 3/5] END .......................alpha=0.001;, score=0.934 total time=   7.5s
[CV 4/5] END .......................alpha=0.001;, score=0.942 total time=  10.0s
[CV 5/5] END .......................alpha=0.001;, score=0.946 total time=   9.7s
[CV 1/5] END ........................alpha=0.01;, score=0.443 total time=   5.4s
[CV 2/5] END ........................alpha=0.01;, score=0.551 total time=   6.7s
[CV 3/5] END ........................alpha=0.01;, score=0.934 total time=   6.7s
[CV 4/5] END ........................alpha=0.01;, score=0.942 total time=   7.8s
[CV 5/5] END ........................alpha=0.01;, score=0.946 total time=   9.0s
[CV 1/5] END .........................alpha=0.1;, score=0.443 total time=   5.5s
[CV 2/5] END .........................alpha=0.1;,

Lasso(alpha=100)
0.7631745091671739
{'alpha': 100}
CPU times: user 25min 16s, sys: 1min 36s, total: 26min 52s
Wall time: 3min 45s


In [28]:
%%time
rf_cv = GridSearchCV(RandomForestRegressor(),
                     {'n_estimators':[10, 100, 1000, 2000], 'max_depth':[1, 3, 5, 10]},
                     cv=5, verbose=3)
rf_cv.fit(X_train, y_train)
show_prediction(df_test, rf_cv.best_estimator_.predict(X_test), counter_country)
print(rf_cv.best_estimator_)
print(rf_cv.best_score_)
print(rf_cv.best_params_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END .....max_depth=1, n_estimators=10;, score=-0.030 total time=   0.1s
[CV 2/5] END .....max_depth=1, n_estimators=10;, score=-0.002 total time=   0.1s
[CV 3/5] END .....max_depth=1, n_estimators=10;, score=-0.026 total time=   0.1s
[CV 4/5] END .....max_depth=1, n_estimators=10;, score=-0.186 total time=   0.1s
[CV 5/5] END .....max_depth=1, n_estimators=10;, score=-0.658 total time=   0.1s
[CV 1/5] END ....max_depth=1, n_estimators=100;, score=-0.030 total time=   0.6s
[CV 2/5] END ....max_depth=1, n_estimators=100;, score=-0.002 total time=   0.5s
[CV 3/5] END ....max_depth=1, n_estimators=100;, score=-0.025 total time=   0.6s
[CV 4/5] END ....max_depth=1, n_estimators=100;, score=-0.196 total time=   0.5s
[CV 5/5] END ....max_depth=1, n_estimators=100;, score=-0.667 total time=   0.5s
[CV 1/5] END ...max_depth=1, n_estimators=1000;, score=-0.030 total time=   4.7s
[CV 2/5] END ...max_depth=1, n_estimators=1000;,

RandomForestRegressor(max_depth=10, n_estimators=10)
CPU times: user 15min 29s, sys: 1.3 s, total: 15min 31s
Wall time: 16min 12s


In [20]:
%%time
rf_cv = GridSearchCV(RandomForestRegressor(random_state=100),
                     {'n_estimators':[5, 10, 15, 20, 50, 100], 'max_depth':[5, 10, 15]},
                     cv=5, verbose=3)
rf_cv.fit(X_train, y_train)
show_prediction(df_test, rf_cv.best_estimator_.predict(X_test), counter_country)
print(rf_cv.best_estimator_)
print(rf_cv.best_score_)
print(rf_cv.best_params_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END .......max_depth=5, n_estimators=5;, score=0.486 total time=   0.1s
[CV 2/5] END .......max_depth=5, n_estimators=5;, score=0.650 total time=   0.1s
[CV 3/5] END .......max_depth=5, n_estimators=5;, score=0.978 total time=   0.1s
[CV 4/5] END .......max_depth=5, n_estimators=5;, score=0.989 total time=   0.1s
[CV 5/5] END .......max_depth=5, n_estimators=5;, score=0.990 total time=   0.1s
[CV 1/5] END ......max_depth=5, n_estimators=10;, score=0.486 total time=   0.1s
[CV 2/5] END ......max_depth=5, n_estimators=10;, score=0.650 total time=   0.1s
[CV 3/5] END ......max_depth=5, n_estimators=10;, score=0.979 total time=   0.1s
[CV 4/5] END ......max_depth=5, n_estimators=10;, score=0.990 total time=   0.2s
[CV 5/5] END ......max_depth=5, n_estimators=10;, score=0.990 total time=   0.2s
[CV 1/5] END ......max_depth=5, n_estimators=15;, score=0.486 total time=   0.2s
[CV 2/5] END ......max_depth=5, n_estimators=15;

RandomForestRegressor(max_depth=10, random_state=100)
0.8263037759655937
{'max_depth': 10, 'n_estimators': 100}
CPU times: user 1min 17s, sys: 606 ms, total: 1min 18s
Wall time: 1min 18s


In [43]:
%%time
rf_cv = GridSearchCV(RandomForestRegressor(random_state=100),
                     {'n_estimators':[5, 10, 15, 20, 50, 100], 'max_depth':[5, 10, 15]},
                     cv=5, verbose=3)
rf_cv.fit(X_train, y_train)
show_prediction(df_test, rf_cv.best_estimator_.predict(X_test), counter_country)
print(rf_cv.best_estimator_)
print(rf_cv.best_score_)
print(rf_cv.best_params_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END .......max_depth=5, n_estimators=5;, score=0.472 total time=   0.1s
[CV 2/5] END .......max_depth=5, n_estimators=5;, score=0.771 total time=   0.1s
[CV 3/5] END .......max_depth=5, n_estimators=5;, score=0.984 total time=   0.1s
[CV 4/5] END .......max_depth=5, n_estimators=5;, score=0.990 total time=   0.1s
[CV 5/5] END .......max_depth=5, n_estimators=5;, score=0.992 total time=   0.1s
[CV 1/5] END ......max_depth=5, n_estimators=10;, score=0.472 total time=   0.2s
[CV 2/5] END ......max_depth=5, n_estimators=10;, score=0.771 total time=   0.2s
[CV 3/5] END ......max_depth=5, n_estimators=10;, score=0.984 total time=   0.2s
[CV 4/5] END ......max_depth=5, n_estimators=10;, score=0.990 total time=   0.2s
[CV 5/5] END ......max_depth=5, n_estimators=10;, score=0.992 total time=   0.2s
[CV 1/5] END ......max_depth=5, n_estimators=15;, score=0.472 total time=   0.2s
[CV 2/5] END ......max_depth=5, n_estimators=15;

RandomForestRegressor(max_depth=10, random_state=100)
0.8475580267765069
{'max_depth': 10, 'n_estimators': 100}
CPU times: user 1min 20s, sys: 792 ms, total: 1min 21s
Wall time: 1min 22s


In [44]:
pickle.dump(rf_cv.best_estimator_, open('../data/final/rf_2211261626.pkl','wb'))

In [23]:
estimator = pickle.load(open('../data/final/rf_2211261400.pkl','rb'))

In [25]:

show_prediction(df_test, estimator.predict(X_test), counter_country)